## Daysim Person and Household Models

*Summaries:*
    - Total People & Workers
    - Household Size
    - Worker Types
    - Distance to Work
    - Transit Pass Ownership
    - Age Distribution
    - Vehicle Ownership
---

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import os
import numpy as np
import pandas as pd
import h5py
import pylab as P
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

# Show charts in notebook
%pylab inline

In [ ]:
# Set main model directory to parent directory

# 2006 survey
model_dir = r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize'

# 2014 survey
base_dir = r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize' 

In [ ]:
# Read Model Scenario Results
scen = h5py.File(model_dir + r'/survey.h5','r+')
scen_name = '2006 Survey'

In [ ]:
# Read Base Data
base_file = r'/survey14.h5'

base = h5py.File(base_dir + base_file ,'r+')
base_name = '2014 Survey'

In [ ]:
# Get person data into a single dataframe for base and scen
data_table = 'Person'

person_scen = pd.DataFrame(data={'Household ID': [i[0] for i in scen[data_table]['hhno'][:]],
                               'Person Number': [i[0] for i in scen[data_table]['pno'][:]],
                               'Transit Pass': [i[0] for i in scen[data_table]['ptpass'][:]],
                                'Auto Time to Work': [i[0] for i in scen[data_table]['pwautime'][:]],
                                'Auto Distance to Work': [i[0] for i in scen[data_table]['pwaudist'][:]],
                                'Worker Type': [i[0] for i in scen[data_table]['pwtyp'][:]],
                                'Usual Commute Mode': [i[0] for i in scen[data_table]['pwtaz'][:]],
                                'Workplace TAZ': [i[0] for i in scen[data_table]['pwtaz'][:]],
                                'Age': [i[0] for i in scen[data_table]['pagey'][:]],
                                'Expansion Factor': [i[0] for i in scen[data_table]['psexpfac'][:]]})

person_base = pd.DataFrame(data={'Household ID': [i[0] for i in base[data_table]['hhno'][:]],
                                 'Person Number': [i[0] for i in base[data_table]['pno'][:]],
                                 'Transit Pass': [i[0] for i in base[data_table]['ptpass'][:]],
                                 'Auto Time to Work': [i[0] for i in base[data_table]['ptpass'][:]],
                                 'Auto Distance to Work': [i[0] for i in base[data_table]['pwautime'][:]],
                                 'Worker Type': [i[0] for i in base[data_table]['pwtyp'][:]],
                                 'Usual Commute Mode': [i[0] for i in base[data_table]['puwmode'][:]],
                                 'Workplace TAZ': [i[0] for i in base[data_table]['pwtaz'][:]],
                                 'Age': [i[0] for i in base[data_table]['pagey'][:]],
                                 'Expansion Factor': [i[0] for i in base[data_table]['psexpfac'][:]]})

In [ ]:
# Create unique ID for person by concatenating household ID and person number 
person_scen['personID'] = (person_scen['Household ID'].astype('str')+person_scen['Person Number'].astype('str')).astype('int')
person_base['personID'] = (person_base['Household ID'].astype('str')+person_base['Person Number'].astype('str')).astype('int')

In [ ]:
# Get household data into a single dataframe for base and scen
data_table = 'Household'

hh_scen = pd.DataFrame(data={'Household ID': [i[0] for i in scen[data_table]['hhno'][:]],
                                'Household Size': [i[0] for i in scen[data_table]['hhsize'][:]],
                                'Household Vehicles': [i[0] for i in scen[data_table]['hhvehs'][:]],
                                 'Household Workers': [i[0] for i in scen[data_table]['hhwkrs'][:]],
                                 'Household Income': [i[0] for i in scen[data_table]['hhincome'][:]],
                                 'Household TAZ': [i[0] for i in scen[data_table]['hhtaz'][:]],
                                 'Expansion Factor': [i[0] for i in scen[data_table]['hhexpfac'][:]]})

hh_base = pd.DataFrame(data={'Household ID': [i[0] for i in base[data_table]['hhno'][:]],
                                 'Household Size': [i[0] for i in base[data_table]['hhsize'][:]],
                                 'Household Vehicles': [i[0] for i in base[data_table]['hhvehs'][:]],
                                 'Household Workers': [i[0] for i in base[data_table]['hhwkrs'][:]],
                                 'Household Income': [i[0] for i in base[data_table]['hhincome'][:]],
                                 'Household TAZ': [i[0] for i in base[data_table]['hhtaz'][:]],
                                 'Expansion Factor': [i[0] for i in base[data_table]['hhexpfac'][:]]})

In [ ]:
# Add labels for worker type
labels = {
  0: "Not a worker",  
  1: "Full-time worker",
  2: "Part-time worker",
}

person_base['Worker Type'] = ([labels[x] for x in person_base['Worker Type']])
person_scen['Worker Type'] = ([labels[x] for x in person_scen['Worker Type']])

In [ ]:
# Join household records to person records
hh_per_scen = person_scen.join(hh_scen,on='Household ID',lsuffix='_p',rsuffix='_h')

In [ ]:
# Household TAZ

In [ ]:
# Join geography info to household TAZ location
taz_lookup = pd.read_csv(r'utils/taz_lookup.csv')
hh_per_scen = hh_per_scen.merge(taz_lookup, left_on='Household TAZ',right_on='TAZ')

In [ ]:
hh_per_scen

## Person and Worker Overview

**Total People**

In [ ]:
print scen_name + ": "+ str(person_scen['personID'].count()).split('.')[0]
print base_name + ": "+ str(person_base['personID'].count()).split('.')[0]

**Average Household Size**

In [ ]:
print scen_name + ": "+ str(hh_scen['Household Size'].mean())
print base_name + ": "+ str(hh_base['Household Size'].mean())

**Household Size Distribution**

In [ ]:
df = pd.DataFrame([hh_scen.groupby('Household Size').count()['Household ID'],
                   hh_base.groupby('Household Size').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df

In [ ]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/person_scen.count()['Household ID'],
                   df[base_name]/person_base.count()['Household ID']]).T
df_dist
display_charts(df_dist, kind='bar', title='Household Size Distribution')

**Workers Types**

In [ ]:
df = pd.DataFrame([person_scen.groupby('Worker Type').count()['personID'],
                   person_base.groupby('Worker Type').count()['personID']]).T
df.columns=([scen_name,base_name])
df

** Distribution of Worker Types**

In [ ]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/person_scen.count()['personID'],
                   df[base_name]/person_base.count()['personID']]).T

In [ ]:
display_charts(df_dist, kind='bar', title='Worker Type Distribution')

In [ ]:
# Workers at Home

**Workers by County**

In [ ]:
# Workers by County
hh_per_scen.groupby

## Distance to Work

In [ ]:
# Filter out non-workers
scen_df = (person_scen[person_scen['Worker Type'] <> 'Not a worker'])
base_df = (person_base[person_base['Worker Type'] <> 'Not a worker'])

df = pd.DataFrame([scen_df.groupby('Age').mean()['Auto Distance to Work'],
              base_df.groupby('Age').mean()['Auto Distance to Work']]).T
df.columns=([scen_name,base_name])
df.fillna(0,inplace=True)
df
# person_scen

# Transit Pass Ownership

In [ ]:
df = pd.DataFrame([person_scen.groupby('Transit Pass').mean()['Auto Distance to Work'],
              person_base.groupby('Transit Pass').mean()['Auto Distance to Work']]).T
df.columns=([scen_name,base_name])

In [ ]:
person_base['Transit Pass'].mean()

In [ ]:
df

In [ ]:
person_base['Transit Pass'];
base_name

In [ ]:
person_base.groupby('Transit Pass').mean()['Auto Distance to Work']

# Age Distribution

In [ ]:
# 2006 survey age is continuous variable
# 2014 is categorical
from collections import OrderedDict


# lookup for 2014 names (which are averaged based on min and max of bin)
age_cat = OrderedDict([(2, '<5'),
           (8, ' 5-11'),
           (14, '12-15'),
           (17, '16-17'),
          (21, '18-24'),
          (30, '25-34'),
          (40, '35-44'),
          (50, '44-54'),
          (60, '55-64'),
          (70, '65-74'),
          (80, '75-84'),
          (90, '85+')])


person_base['Age'] = [age_cat[x] for x in person_base['Age'].as_matrix()]

In [ ]:
# convert 2006 into categorical
age_bins = [0,4,11,15,17,24,34,44,54,64,74,84,1000]
person_scen['Age'] = pd.cut(person_scen['Age'],age_bins,labels=age_cat.values())


In [ ]:
df = pd.DataFrame([person_scen.groupby('Age').count()['Household ID'],
                   person_base.groupby('Age').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df

In [ ]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

display_charts(df_new, title='Age', kind='bar')

# Auto Ownership

In [ ]:
new_hh_scen = hh_scen[hh_scen.index>=0]
df = pd.DataFrame([new_hh_scen.groupby('Household Vehicles').count()['Household ID'],
                   hh_base.groupby('Household Vehicles').count()['Household ID']]).T
df.columns=([scen_name,base_name])
df=df[df.index>=0]
df

In [ ]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T

In [ ]:
display_charts(df_new, title='Vehicles per Household', kind='bar', xlim=(0,6))